In [ ]:
using Pkg
Pkg.activate("."); Pkg.instantiate()

In [ ]:
using Pkg
Pkg.update()


In [ ]:
#import Pkg
Pkg.add("AxisArrays")
Pkg.add("JuMP")
Pkg.add("XLSX")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("AxisArrays")
Pkg.add("Statistics")
Pkg.add("PGFPlotsX")
Pkg.add("AxisArrays")
Pkg.add("CPLEX")
Pkg.add("HiGHS")
Pkg.add("CSV")
Pkg.add("Tables")

In [ ]:
using JuMP, XLSX, DataFrames, Plots, AxisArrays, Statistics, PGFPlotsX, CPLEX, HiGHS, CSV, Tables

In [ ]:
pgfplotsx()

# Read In And Prepare Data 

In [ ]:
mydata = Dict{String, DataFrame}()

In [ ]:
xl = XLSX.readxlsx("data/curtailmentinfonew.xlsx")
mydata["curtailmentinfo"] = DataFrame(XLSX.gettable(xl["curtailmentinfo"]; infer_eltypes=true))
print("curtailment data read in successfully")

In [ ]:
mydata["curtailmentinfo"]

In [ ]:
xl = XLSX.readxlsx("data/generatorinfo.xlsx")
generatorinfo = XLSX.sheetnames(xl)
mydata["generatorinfo"] = DataFrame(XLSX.gettable(xl["generatorinfo"]; infer_eltypes=true))
print("generator data read in successfully")

In [ ]:
mydata["generatorinfo"]

In [ ]:
xl = XLSX.readxlsx("data/initdemand.xlsx")
initdemand = XLSX.sheetnames(xl)
mydata["initdemand"] = DataFrame(XLSX.gettable(xl["initdemand"]; infer_eltypes=true))
print("demand data read in successfully")

In [ ]:
mydata["initdemand"]

In [ ]:
xl = XLSX.readxlsx("data/lineconstraints.xlsx")
linecontraints = XLSX.sheetnames(xl)
mydata["lineconstraints"] = DataFrame(XLSX.gettable(xl["lineconstraints"]; infer_eltypes=true))
print("line constraint data read in successfully")

In [ ]:
mydata["lineconstraints"]

In [ ]:
xl = XLSX.readxlsx("data/nodeinfo.xlsx")
nodeinfo = XLSX.sheetnames(xl)
mydata["nodeinfo"] = DataFrame(XLSX.gettable(xl["nodeinfo"]; infer_eltypes=true))
print("node data read in successfully")

In [ ]:
mydata["nodeinfo"]

In [ ]:
xl = XLSX.readxlsx("data/shiftinginfonew.xlsx")
shiftinginfo = XLSX.sheetnames(xl)
mydata["shiftinginfo"] = DataFrame(XLSX.gettable(xl["shiftinginfo"]; infer_eltypes=true))
print("shifting information read in successfully")

In [ ]:
mydata["shiftinginfo"]

In [ ]:
xl = XLSX.readxlsx("data/windfarminfo.xlsx")
windfarminfo = XLSX.sheetnames(xl)
mydata["windfarminfo"] = DataFrame(XLSX.gettable(xl["windfarminfo"]; infer_eltypes=true))
print("windfarm data read in successfully")

In [ ]:
mydata["windfarminfo"]

In [ ]:
xl = XLSX.readxlsx("data/windscenarios.xlsx")
windscenarios = XLSX.sheetnames(xl)
mydata["windscenarios"] = DataFrame(XLSX.gettable(xl["windscenarios"]; infer_eltypes=true))
print("wind scenario data read in successfully")


In [ ]:
mydata["windscenarios"]

### Set generation

In [ ]:
G = mydata["generatorinfo"][:, :ID]
L = mydata["lineconstraints"][:, :ID]
T = mydata["initdemand"][:, :t]          #times
N = mydata["nodeinfo"][:, :ID]
S = unique(mydata["windscenarios"][:, :scenario])
K = unique(mydata["curtailmentinfo"][:, :k])

NG = length(G)
NL = length(L)
NT = length(T)
NN = length(N)
NS = length(S)
NK = length(K)

# Prepare data

Prepare the generator information

In [ ]:
#generatorinfo
generators = Dict{String, AxisArray}()
for c in names(mydata["generatorinfo"])
    help_array1 = AxisArray(mydata["generatorinfo"][:, c], g=G)
    # Assign the 'help_array1' AxisArray to the 'c' key in the 'generators' dictionary
    generators[c] = help_array1
end

Prepare all of the line constraints

In [ ]:
#lineconstraints
#lines = Dict(c => AxisArray(mydata["lineconstraints"][:, c], l=L) for c in names(mydata["lineconstraints"]));
lines = Dict{String, AxisArray}()
for c in names(mydata["lineconstraints"])
    help_array2 = AxisArray(mydata["lineconstraints"][:, c], l=L)
    lines[c] = help_array2
end
B = AxisArray(zeros(NL, NN), l=L, n=N)
for l in L
    from_node = lines["from"][l]
    to_node = lines["to "][l]
    X = lines["X"][l]
    # Loop through each node.
    for (idn, node) in enumerate(N)
        if node == from_node
            B[l, idn] = -1/X
        elseif node == to_node
            B[l, idn] = 1/X
        end
    end
end 


Prepare the data of the initial demand (demand before DR)

In [ ]:
#initdemand
load = AxisArray(Matrix(mydata["initdemand"][:, Not(:t)]), t=T, n=N)
cost_ls = 100.0

Prepare the load shifiting information

In [ ]:
#shiftinginfo
cols = names(mydata["shiftinginfo"][:, Not(["t", "n", "k"])])
#DRSd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)

DRSd = Dict{String, AxisArray}()
for c in cols
# Loop through the column names and create a new AxisArray for each column
    help_array3 = AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K)
    DRSd[c] = help_array3
end
# Loop through each row in "shiftinginfo" and assign the values to the corresponding AxisArray in DRSd
for r in eachrow(mydata["shiftinginfo"])
    for c in cols
        DRSd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end

Prepare the load curtailment information

In [ ]:
#curtailmentinfo
cols = names(mydata["curtailmentinfo"][:, Not(["t", "n", "k"])])

#DRCd = Dict(c => AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K) for c in cols)
DRCd = Dict{String, AxisArray}()
for c in cols
    help_array4 = AxisArray(zeros(NT, NN, NK), t=T, n=N, k=K)
    DRCd[c] = help_array4
end
# Iterate over each row in the curtailmentinfo data
for r in eachrow(mydata["curtailmentinfo"])
    for c in cols
        DRCd[c][r[:t], r[:n], r[:k]] = r[c]
    end
end


In [ ]:
DRCd

Prepare the information of the location of the nodes and generators

In [ ]:
#location

# Create an AxisArray where ng[n,g] = 1 if generator g is located at node n, and 0 otherwise
ng = AxisArray(
    [[g for g in G if generators["location"][g] == idn] for (idn, n) in enumerate(N)],
    n = N
)
#node

# Create an AxisArray where nl[n,l] = 1 if line l originates at node n, -1 if it terminates at node n, and 0 otherwise
nl = AxisArray(zeros(Int, NN, NL), n=N, l=L)
for l in L
    nl[N[lines["from"][l]], l] = 1
    nl[N[lines["to "][l]], l] = -1
end

In [ ]:
windCap = AxisArray(zeros(NN), n=N)
windCapDict = Dict("n3" => 500.0, "n5" => 500.0, "n16" => 300.0, "n21" => 300.0)

# populate the windCap AxisArray with the capacities 
windCap["n3"] = 500.0
windCap["n5"] = 500.0
windCap["n16"] = 300.0
windCap["n21"] = 300.0

pi = AxisArray(ones(NS)/NS, s=S)

wind = AxisArray(zeros(NS, NN, NT), s=S, n=N, t=T)
# populate the wind array with the wind power generation data from the input data, scale
for r in eachrow(mydata["windscenarios"])
    for t in T
        wind[r[:scenario], r[:node], t] = r[t]*windCap[r[:node]]
    end
end

estwind = mean(wind, dims=1)[1, :, :]
stdwind = std(wind, dims=1)[1, :, :]
cost_ws = 20.0


In [ ]:
pi

In [ ]:
# total_cost = 0.0
# for t in T
#     generator_cost = 0.0
#     reserve_cost = 0.0
#     start_cost = 0.0
    
#     # Generator cost
#     for g in G
#         generator_cost += PGen[g,t] * generators["cost"][g] +
#                             r_up[g,t] * generators["upresercost"][g] +
#                             r_down[g,t] * generators["dnresercost"][g] +
#                             suc[findfirst(isequal(t), T),g]
#     end
    
#     # Reserve cost
#     for n in N
#         reserve_cost += DRC[t, n] + DRS[t, n]
#     end
    
#     # Startup cost
#     for s in S
#         startup_cost = 0.0
#         for g in G
#             startup_cost += generators["cost"][g] * (r_ups[s,g,t]-r_downs[s,g,t])
#         end
#         start_cost += pi[s] * (startup_cost + sum(cost_ws * Wspill[s,n,t] + cost_ls * Lshed[s,n,t] for n in N))
#     end
    
#     # Total cost
#     total_cost += generator_cost + reserve_cost + start_cost
# end


# Model

Variables: 

pgen: the scheduled generation output for each generator g at each time t

u: the state of each generator g (on or off) at each time t

θ0: the phase angles for each node n in the power network during the scheduling phase

θ1: the phase angles for each node n during the balancing phase and each scenario s

Wsc: the scheduled wind power for each node n at each time t

r_up and r_down: the up and down regulation reserves for each generator g at each time t

r_ups and r_downs: the actual up and down regulation provided by each generator g at each time t

Lshed: the amount of load shedding at each node n during each scenario s and time t

Wspill: the amount of wind spillage at each node n during each scenario s and time t

suc: the start-up/shut-down costs for each generator g at each time t


Expressions: 
DRC, DRS, DBC, DBS: the costs/profits associated with demand response contracts
pflow0: the power flow on each transmission line during the scheduling phase
pflow1: the power flow on each transmission line during the balancing phase and each scenario s


Objective is to minimize totalcost
totalcost: the total cost of the system, including the cost of generation, reserves, start-up/shut-down, load shedding, wind spillage, and demand response

The model is intended to represent an electric power system with wind power and demand response (DR) programs.



Generator Limits:
a. The power generated by a generator at time t, plus any upward regulation, should be less than or equal to the generator's capacity times its availability.
b. The upward regulation should be less than or equal to the generator's maximum upward reserve capacity.
c. The downward regulation should be less than or equal to the generator's maximum downward reserve capacity.

Reserve Limits:
The net power generation from all generators connected to a node, plus the power injection from any energy storage system connected to the node, plus the scheduled curtailment of any DR resource, should be equal to the total demand at that node plus any power flow into the node from transmission lines.

Transmission Line Limits:
The power flow through a transmission line at time t should be less than or equal to the line's capacity, and greater than or equal to the negative of the line's capacity.



DR Curtailment Constraints (between ISO and DR aggregator):
a. The scheduled curtailment of a DR resource at time t should be less than or equal to the maximum curtailment allowed for that resource multiplied by its availability.
b. The scheduled curtailment of a DR resource at time t should be greater than or equal to the minimum curtailment allowed for that resource multiplied by its availability.


DR Shifting Constraints (between ISO and DR aggregator):
a. The scheduled shifting of a DR resource at time t should be less than or equal to the maximum shifting allowed for that resource multiplied by its availability.
b. The scheduled shifting of a DR resource at time t should be greater than or equal to the minimum shifting allowed for that resource multiplied by its availability.
c. The scheduled reverse shifting of a DR resource at time t should be less than or equal to the maximum reverse shifting allowed for that resource multiplied by its availability.


DR Curtailment Constraints (between DR aggregator and customer):
a. The scheduled curtailment of a DR resource at time t should be less than or equal to the maximum curtailment allowed for that resource multiplied by its availability.
b. The scheduled curtailment of a DR resource at time t should be greater than or equal to the minimum curtailment allowed for that resource multiplied by its availability.


DR Shifting Constraints (between DR aggregator and customer):
a. The scheduled shifting of a DR resource at time t should be less than or equal to the maximum shifting allowed for that resource multiplied by its availability.
b. The scheduled shifting of a DR resource at time t should be greater than or equal to the minimum shifting allowed for that resource multiplied by

In [ ]:
bdr = Model(CPLEX.Optimizer)
set_silent(bdr)

@variable(bdr, PGen[G, T] >= 0) # scheduled generator output for each generator g at each time t
@variable(bdr, u[G, T], Bin) # state of each generator g (on or off) at each time 
@variable(bdr, θ0[N, T]) # the phase angles for each node n in the power network during the scheduling phase
@variable(bdr, θ1[N, S, T]) # the phase angles for each node n in the power network during the balancing phase and each scenario s
@variable(bdr, Wsc[N, T] >= 0)# the scheduled wind power for each node n at each time t 
@variable(bdr, r_up[G, T] >= 0) # up regulation reserves for each generator g at each time t
@variable(bdr, r_down[G, T] >= 0) # down regulation reserves for each generator g at each time t
@variable(bdr, r_ups[S, G, T] >= 0) # actual up regulation provided by each generator g at each time t
@variable(bdr, r_downs[S, G, T] >= 0) # actual down regulation provided by each generator g at each time t
@variable(bdr, Lshed[S, N, T] >= 0) # amount of load shedding at each node n at each time t
@variable(bdr, Wspill[S, N, T] >= 0) # the amount of wind power spilled at each node n at each time t
@variable(bdr, suc[1:NT, G] >= 0) # start-up/shut-down cost for each generator g at each time t

    # DR contracts
    # DR agg <> ISO contract parameters
@variable(bdr, DRCK[T, N, K] >= 0) # DR curtailment blocks
@variable(bdr, DRSK[T, N, K] >= 0) # DR shifting blocks
@variable(bdr, DRC_SU[T, N, K] >= 0) # DRC startup cost
@variable(bdr, DRSRK[T, N, K] >= 0) # DR recovery blocks
@variable(bdr, uDRC[T, N, K], Bin) # DR curtailment active
@variable(bdr, uDRS[T, N, K], Bin) # DR shifting activate
@variable(bdr, yDR[T, N, K], Bin) # DRC activated
@variable(bdr, zDR[T, N, K], Bin) # DRC de-activated
@variable(bdr, uDRSR[T, N, K], Bin) # DRS recovery active
    # same for DR agg <> cust. contracts
@variable(bdr, DBCK[T, N, K] >= 0)
@variable(bdr, DBSK[T, N, K] >= 0)
@variable(bdr, DBC_SU[T, N, K] >= 0) 
@variable(bdr, DBSRK[T, N, K] >= 0)
@variable(bdr, uDBC[T, N, K], Bin)
@variable(bdr, uDBS[T, N, K], Bin)
@variable(bdr, yDB[T, N, K], Bin)
@variable(bdr, zDB[T, N, K], Bin)
@variable(bdr, uDBSR[T, N, K], Bin)
    # Dual multipliers/linearization variables for the Karush-Kuhn-Tucker conditions
@variable(bdr, Α[T,N,K] >= 0)
@variable(bdr, Β[T,N,K] >= 0)
@variable(bdr, Δ[T,N,K] >= 0)
@variable(bdr, Ε[T,N,K] >= 0)
@variable(bdr, Ζ[T,N,K] >= 0)
@variable(bdr, Η[T,N,K] >= 0)
@variable(bdr, Ι[T,N,K] >= 0)
@variable(bdr, Κ[T,N,K] >= 0)
@variable(bdr, Λ[T,N,K] >= 0)
@variable(bdr, Μ[T,N,K] >= 0)


# fix slack bus
fix.(θ0["n1", :], 0);
fix.(θ1["n1", :, :], 0);


@expression(bdr, DRC[t in T, n in N], sum(DRCd["cost"][t,n,k]*DRCK[t, n, k]+DRC_SU[t,n,k] for k in K))
@expression(bdr, DRS[t in T, n in N], sum(DRSd["cost"][t,n,k]*DRSK[t, n, k] for k in K))
@expression(bdr, DBC[t in T, n in N], sum(DRCd["costb"][t,n,k]*DBCK[t, n, k] for k in K))
@expression(bdr, DBS[t in T, n in N], sum(DRSd["costb"][t,n,k]*DBSK[t, n, k] for k in K))
@expression(bdr, pflow0[l in L, t in T], sum(B[l,n]*θ0[n,t] for n in N))
@expression(bdr, pflow1[l in L, s in S, t in T], sum(B[l,n]*θ1[n,s,t] for n in N))

# Generator Limits 

@constraint(bdr, pup[g in G, t in T], PGen[g, t] + r_up[g, t] <= generators["cap"][g] * u[g,t])

@constraint(bdr, res_up[g in G, t in T], r_up[g, t] <= generators["maxupresercap"][g])

@constraint(bdr, res_dn[g in G, t in T], r_down[g, t] <= generators["maxdnresercap"][g])

# Equality constraint

@constraint(bdr, bal0[n in N, t in T], sum(PGen[g,t] for g in ng[n]) + Wsc[n,t] + sum(DRCK[t,n,k] + DRSK[t,n,k] - DRSRK[t,n,k] for k in K) == load[t,n] + sum(pflow0[l, t]*nl[n,l] for l in L))

# Transmission Line Limits

@constraint(bdr, pfmax0[l in L, t in T], pflow0[l, t] <= lines["flowcap"][l])

@constraint(bdr, pfmin0[l in L, t in T], pflow0[l, t] >= -lines["flowcap"][l])

# DR curtailment constraints between ISO and Aggregator

@constraint(bdr, drcmax[t in T, n in N, k in K], DRCK[t,n,k] <= uDRC[t,n,k] * DRCd["max"][t,n,k]  )

@constraint(bdr, drcmin[t in T, n in N, k in K], DRCK[t,n,k] >= uDRC[t,n,k] * DRCd["min"][t,n,k])

@constraint(bdr, drcst1[τ in 2:NT, n in N, k in K], yDR[T[τ],n,k] - zDR[T[τ],n,k] == uDRC[T[τ],n,k] - uDRC[T[τ-1],n,k])

@constraint(bdr, drcst2[t in T, n in N, k in K], yDR[t,n,k] + zDR[t,n,k] <= 1)

@constraint(bdr, drcst3[t in T, n in N, k in K], DRC_SU[t,n,k] >= DRCd["SUC"][t,n,k] * yDR[t,n,k])

# DR shifting constraints between ISO and Aggregator

@constraint(bdr, drsmax[t in T, n in N, k in K], DRSK[t,n,k] <= uDRS[t,n,k] * DRSd["max"][t,n,k])

@constraint(bdr, drsmin[t in T, n in N, k in K], DRSK[t,n,k] >= uDRS[t,n,k] * DRSd["min"][t,n,k])

@constraint(bdr, drsmxr[t in T, n in N, k in K], DRSRK[t,n,k] <= 2*uDRSR[t,n,k])

@constraint(bdr, drsorr[t in T, n in N, k in K], uDRS[t,n,k] + uDRSR[t,n,k] <= 1)

@constraint(bdr, drbbal[n in N, k in K], sum(DRSRK[t,n,k] for t in T) == sum(DRSK[t,n,k] for t in T))

# DR curtailment constraints between Aggregator and Customer

@constraint(bdr, dbcmax[t in T, n in N, k in K], DBCK[t,n,k] <= uDRC[t,n,k] * DRCd["max"][t,n,k])

@constraint(bdr, dbcmin[t in T, n in N, k in K], DBCK[t,n,k] >= uDRC[t,n,k] * DRCd["min"][t,n,k])

@constraint(bdr, dbcst1[τ in 2:NT, n in N, k in K], yDB[T[τ],n,k] - zDB[T[τ],n,k] == uDBC[T[τ],n,k] - uDBC[T[τ-1],n,k])

@constraint(bdr, dbcst2[t in T, n in N, k in K], yDB[t,n,k] + zDB[t,n,k] <= 1)

@constraint(bdr, dbcst3[t in T, n in N, k in K], DBC_SU[t,n,k] >= DRCd["SUCb"][t,n,k] * yDB[t,n,k])

# DR shifting constraints between Aggregator and Customer

@constraint(bdr, dbsmax[t in T, n in N, k in K], DBSK[t,n,k] <= uDBS[t,n,k] * DRSd["max"][t,n,k])

@constraint(bdr, dbsmin[t in T, n in N, k in K], DBSK[t,n,k] >= uDBS[t,n,k] * DRSd["min"][t,n,k])

@constraint(bdr, dbsmxr[t in T, n in N, k in K], DBSRK[t,n,k] <= 2*uDBSR[t,n,k])

@constraint(bdr, dbsorr[t in T, n in N, k in K], uDBS[t,n,k] + uDBSR[t,n,k] <= 1)

@constraint(bdr, dbbbal[n in N, k in K], sum(DBSRK[t,n,k] for t in T) == sum(DBSK[t,n,k] for t in T))

# linearization constraints

@constraint(bdr, conAB[t in T, n in N, k in K], Α[t,n,k]-Β[t,n,k] >= DRCd["cost"][t,n,k])

@constraint(bdr, conDE[t in T, n in N, k in K], Δ[t,n,k]-Ε[t,n,k] >= DRSd["cost"][t,n,k])

@constraint(bdr, conHI[t in T, n in N, k in K], Η[t,n,k]-Ι[t,n,k] >= -DRCd["costb"][t,n,k])

@constraint(bdr, conKL[t in T, n in N, k in K], Κ[t,n,k]-Λ[t,n,k] >= -DRSd["costb"][t,n,k])

@constraint(bdr, strong_duality, 
    sum(
        Α[t,n,k]*DRCd["max"][t,n,k] - Β[t,n,k]*DRCd["min"][t,n,k] +
        Δ[t,n,k]*DRSd["max"][t,n,k] - Ε[t,n,k]*DRSd["min"][t,n,k] +
        Ζ[t,n,k]*2 +
        Η[t,n,k]*DRCd["max"][t,n,k] - Ι[t,n,k]*DRCd["min"][t,n,k] +
        Κ[t,n,k]*DRSd["max"][t,n,k] - Λ[t,n,k]*DRSd["min"][t,n,k] +
        Μ[t,n,k]*2
    for t in T for n in N for k in K) ==
    sum(
        DRCd["cost"][t,n,k] * DRCK[t,n,k] +
        DRSd["cost"][t,n,k] * DRSK[t,n,k] - 
        DRCd["costb"][t,n,k] * DBCK[t,n,k] - 
        DRSd["costb"][t,n,k] * DBSK[t,n,k]    
    for t in T for n in N for k in K))

# wind 

@constraint(bdr, resup[s in S, g in G, t in T], r_ups[s,g,t] <= r_up[g,t])

@constraint(bdr, resdn[s in S, g in G, t in T], r_downs[s,g,t] <= r_down[g,t])

@constraint(bdr, bals[s in S, n in N, t in T], 
    sum(r_ups[s,g,t]-r_downs[s,g,t] for g in ng[n]) + 
    wind[s,n,t] - Wsc[n,t] - Wspill[s,n,t] + Lshed[s,n,t] == 
    -sum((pflow1[l,s,t] - pflow0[l,t])*nl[n,l] for l in L))

@constraint(bdr, windspill[s in S, n in N, t in T], Wspill[s,n,t] <= wind[s,n,t])

@constraint(bdr, pfmax1[l in L, s in S, t in T], pflow1[l,s,t] <= lines["flowcap"][l])

@constraint(bdr, pfmin1[l in L, s in S, t in T], pflow1[l,s,t] >= -lines["flowcap"][l])


    
@objective(bdr, Min, 
sum(
    sum(
        PGen[g,t] * generators["cost"][g] +
        r_up[g,t] * generators["upresercost"][g] +
        r_down[g,t] * generators["dnresercost"][g] +
        suc[findfirst(isequal(t), T),g]
    for g in G) +
    sum(DRC[t, n] + DRS[t, n] for n in N) +
    sum(
        pi[s] * (
            sum(generators["cost"][g] * (r_ups[s,g,t]-r_downs[s,g,t]) for g in G) +
            sum(cost_ws * Wspill[s,n,t] + cost_ls * Lshed[s,n,t] for n in N)
            )
    for s in S)
for t in T)
)

bdr

In [ ]:
display("image/png", read("data/Objective Function.png"))

# Model

In [ ]:
optimize!(bdr)

In [ ]:
termination_status(bdr)

In [ ]:
objective_value(bdr)

In [ ]:
LoadBeforeDR = sum(load, dims=2)[:, 1]
for i=1:size(LoadBeforeDR,1)            
    println(LoadBeforeDR[i,:])        
end

In [ ]:
LoadAfterDR = [sum(load[t,:]) - sum(value.(DRCK[t,:,:]) .+ value.(DRSK[t,:,:]) .- value.(DRSRK[t,:,:])) for t in T]
LoadAfterDR
for i=1:size(LoadAfterDR,1)            
    println(LoadAfterDR[i,:])        
end

In [164]:
plot(
    LoadBeforeDR,
    label = "Load Before DR",
    xlabel = "Time",
    ylabel = "Load",
    xlims = (1, 24),
    xticks = 1:24,
)

plot!(
    LoadAfterDR,
    label = "Load After DR",
    xlabel = "Time",
    ylabel = "Load",
    xlims = (1, 24),
    xticks = 1:24,
)